In [2]:
# Example Usage
config_path = '/home/fit/lulei/WORK/xjt/Protein_design/RFdiffusion/mytest/config/inference/link.csv'

# Parse the connection config
connections = parse_connections(config_path)

# --- Test Case ---
B = 2
L = 11
# Sequence: SER(0), GLY(1), GLY(2), GLY(3), GLY(4), ALA(5), GLY(6), GLY(7), ARG(8), GLY(9), ASP(10)
# Connections:
# - SER(0) and ALA(5) are connected by a special bond.
# - ARG(8) and ASP(10) are connected by a special bond.
# - Standard peptide bonds exist between all adjacent residues.
seq_tensor = torch.tensor([[
    aa2num['LYS'],
    21,
    aa2num['GLY'],
    aa2num['GLY'],
    aa2num['GLY'],
    aa2num['ALA'],
    aa2num['ARG'],
    aa2num['GLY'],
    aa2num['GLY'],      
    aa2num['GLY'],
    aa2num['ASP']
]], dtype=torch.long).repeat(B, 1)
rf_index = torch.tensor([0,1,2,3,4,5,6,7,10,11,12]).repeat(B, 1)
# Adjacency matrix
adj_tensor = torch.zeros(B, L, L, dtype=torch.long)
# Connection between SER(0) and ALA(5)
adj_tensor[0, 0, 5] = 1
adj_tensor[0, 5, 0] = 1
# Connection between ARG(8) and ASP(10)
adj_tensor[0, 6, 10] = 1
adj_tensor[0, 10, 6] = 1
adj_tensor[1, 0, 10] = 1
adj_tensor[1, 10, 0] = 1

print("Sequence (numeric):", seq_tensor)
print("Adjacency Matrix:\n", adj_tensor)

# Calculate the distance matrix
distance_matrix = get_dist_matrix(adj_tensor, seq_tensor, connections,dmax=22,rf_index=rf_index)

print("\nCalculated Distance Matrix:\n", distance_matrix)

# --- Verification of a known path ---
# Let's manually trace the path from SER(0)-CA to ALA(5)-CA
# Path 1 (Peptide chain): 5 residues apart = 5 * 3 = 15 bonds
# Path 2 (Special bond): SER(0)[CA]->CB->OG --bond-- ALA(5)[C]->CA. Dist = 2 + 1 + 1 = 4
# The shortest path should be 4.
print(f"\nDistance between SER(0) and ALA(5) should be 4. Calculated: {distance_matrix[0, 0, 5]}")

# Let's trace the path from ARG(8)-CA to ASP(10)-CA
# Path 1 (Peptide chain): 2 residues apart = 2 * 3 = 6 bonds
# Path 2 (Special bond): ARG(8)[CA]->N --bond-- ASP(10)[CG]->CB->CA. Dist = 1 + 1 + 2 = 4
# The shortest path should be 4.
print(f"Distance between ARG(6) and ASP(10) should be 4. Calculated: {distance_matrix[0, 6, 10]}")

# Let's trace from ALA(5)-CA to ARG(8)-CA
# Path (Peptide chain only): 3 residues apart = 3 * 3 = 9 bonds
print(f"Distance between ALA(5) and ARG(6) should be 3. Calculated: {distance_matrix[0, 5, 6]}")


Sequence (numeric): tensor([[11, 21,  7,  7,  7,  0,  1,  7,  7,  7,  3],
        [11, 21,  7,  7,  7,  0,  1,  7,  7,  7,  3]])
Adjacency Matrix:
 tensor([[[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]],

        [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0

In [51]:
import torch
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import shortest_path

# Assuming rfdiffusion.chemical is in the python path
# If not, we might need to add the path
import sys
# Add the parent directory to the path to import rfdiffusion
sys.path.append('/home/fit/lulei/WORK/xjt/Protein_design/RFdiffusion/mytest')
from mytest.rfdiffusion.chemical import aa2num, num2aa, aa_321, aa_123, aabonds, aa2long

# --- Constants for atom names ---
ATOM_CA = "CA"
ATOM_C = "C"
ATOM_N = "N"
BACKBONE_ATOMS = {"N", "CA", "C", "O"}

def parse_connections(config_path):
    """
    Parses the connection configuration from a CSV file.
    Handles multiple connection types for a single residue pair.
    """
    connections = {}
    
    try:
        with open(config_path, 'r') as f:
            next(f)  # Skip header
            for line in f:
                try:
                    parts = line.strip().split(',')
                    if len(parts) < 4: continue
                    res1, res2, atom1, atom2 = [p.strip() for p in parts[:4]]
                    
                    if res1 not in aa2num or res2 not in aa2num:
                        continue

                    res1_num, res2_num = aa2num[res1], aa2num[res2]
                    
                    # Classify connection type
                    is_backbone_conn = atom1 in BACKBONE_ATOMS or atom2 in BACKBONE_ATOMS
                    conn_type = 'backbone' if is_backbone_conn else 'sidechain'
                    
                    conn_tuple = (atom1, atom2, conn_type)

                    # Initialize dict entry if not present
                    if (res1_num, res2_num) not in connections:
                        connections[(res1_num, res2_num)] = []
                    if (res2_num, res1_num) not in connections:
                        connections[(res2_num, res1_num)] = []

                    connections[(res1_num, res2_num)].append(conn_tuple)
                    connections[(res2_num, res1_num)].append((atom2, atom1, conn_type))

                except (ValueError, IndexError):
                    # Silently skip malformed lines
                    continue
    except FileNotFoundError:
        # This is a valid case if no special connections are needed.
        print(f"Info: Connection config file not found at {config_path}. Continuing without special connections.")
        return {}
            
    return connections

def get_dist_matrix(adj, seq, connections, rf_index, dmax=20, N_connect_idx=None, C_connect_idx=None):
    """
    Computes the distance matrix based on covalent bonds using Scipy for performance.
    """
    B, L = adj.shape[:2]
    dist_matrix = torch.full((B, L, L), float(dmax), dtype=torch.float32)
    
    GLY_IDX = aa2num['GLY']
    MASK_IDX = 21

    def is_chain_terminus(b, i, rf_index_b_set):
        res_idx = rf_index[b, i].item()
        is_n_term = (res_idx - 1) not in rf_index_b_set
        is_c_term = (res_idx + 1) not in rf_index_b_set
        return is_n_term or is_c_term

    for b in range(B):
        seq_b = seq[b].clone()
        seq_b[seq_b == MASK_IDX] = GLY_IDX
        
        rf_index_b_set = set(rf_index[b].tolist())

        atom_to_node = {}
        node_counter = 0
        
        # 1. Map all heavy atoms to integer node indices
        for i in range(L):
            res_type = seq_b[i].item()
            atoms_in_res = {atom.strip() for bond in aabonds[res_type] for atom in bond}
            for atom_name in atoms_in_res:
                if (i, atom_name) not in atom_to_node:
                    atom_to_node[(i, atom_name)] = node_counter
                    node_counter += 1
        
        num_nodes = node_counter
        if num_nodes == 0: continue

        row, col = [], []

        # Helper to add edges to the adjacency list
        def add_edge(u, v):
            row.extend([u, v])
            col.extend([v, u])

        # 2. Add intra-residue and peptide bonds
        for i in range(L):
            # Intra-residue
            res_type = seq_b[i].item()
            for atom1, atom2 in aabonds[res_type]:
                u = atom_to_node.get((i, atom1.strip()))
                v = atom_to_node.get((i, atom2.strip()))
                if u is not None and v is not None:
                    add_edge(u, v)
            # Peptide
            if i < L - 1 and rf_index[b, i+1] == rf_index[b, i] + 1:
                u = atom_to_node.get((i, ATOM_C))
                v = atom_to_node.get((i + 1, ATOM_N))
                if u is not None and v is not None:
                    add_edge(u, v)

        # 3. Add special connections from the adjacency matrix
        for i_idx, j_idx in torch.argwhere(adj[b] == 1):
            i, j = i_idx.item(), j_idx.item()
            if i >= j: continue

            res_i_type, res_j_type = seq_b[i].item(), seq_b[j].item()
            atom_i_name, atom_j_name = None, None

            is_i_N_term = N_connect_idx is not None and i == N_connect_idx[b]
            is_i_C_term = C_connect_idx is not None and i == C_connect_idx[b]
            is_j_N_term = N_connect_idx is not None and j == N_connect_idx[b]
            is_j_C_term = C_connect_idx is not None and j == C_connect_idx[b]

            if (is_i_C_term and is_j_N_term) or (is_i_N_term and is_j_C_term):
                atom_i_name = ATOM_C if is_i_C_term else ATOM_N
                atom_j_name = ATOM_N if is_j_N_term else ATOM_C
            else:
                preferred_conn_type = 'backbone' if (is_i_N_term or is_i_C_term or is_j_N_term or is_j_C_term) else 'sidechain'
                possible_conns = connections.get((res_i_type, res_j_type), [])
                if not possible_conns: continue
                
                filtered_conns = [c for c in possible_conns if c[2] == preferred_conn_type]
                conn_to_use = filtered_conns[0] if filtered_conns else possible_conns[0]
                atom_i_name, atom_j_name, _ = conn_to_use

            if atom_i_name and atom_j_name:
                if (atom_i_name in BACKBONE_ATOMS and not is_chain_terminus(b, i, rf_index_b_set)) or \
                   (atom_j_name in BACKBONE_ATOMS and not is_chain_terminus(b, j, rf_index_b_set)):
                    continue
                u = atom_to_node.get((i, atom_i_name))
                v = atom_to_node.get((j, atom_j_name))
                if u is not None and v is not None:
                    add_edge(u, v)

        # 4. Build sparse matrix and calculate all-pairs shortest paths
        adj_matrix_sparse = csr_matrix((([1]*len(row)), (row, col)), shape=(num_nodes, num_nodes))
        
        atom_dist_matrix = shortest_path(csgraph=adj_matrix_sparse, directed=False, unweighted=True)

        # 5. Populate the final distance matrix for CA atoms
        for i in range(L):
            dist_matrix[b, i, i] = 0
            node_i = atom_to_node.get((i, ATOM_CA))
            if node_i is None: continue
            for j in range(i + 1, L):
                node_j = atom_to_node.get((j, ATOM_CA))
                if node_j is None: continue
                
                dist = atom_dist_matrix[node_i, node_j]
                final_dist = min(dist, dmax)
                dist_matrix[b, i, j] = final_dist
                dist_matrix[b, j, i] = final_dist
                
    return dist_matrix

In [58]:
# Example Usage
# --- Mock Config File ---
# To make this example self-contained, we'll simulate the config file in memory.
import time

start_time = time.time()
# In a real scenario, you would use the path to your file:
config_path = '/home/fit/lulei/WORK/xjt/Protein_design/RFdiffusion/mytest/config/inference/link.csv'

# Parse the connection config
connections = parse_connections(config_path)
print(connections)
print(len(connections))
# =============================================================================
# --- Test Case 1: Intra-chain connections with chain break and mask ---
# =============================================================================
print("--- Test Case 1: Intra-chain connections with chain break and mask ---")
B, L = 1, 11
# Sequence: LYS(0), MASK(1), GLY(2), GLY(3), GLY(4), ALA(5), ARG(6), GLY(7), GLY(8), GLY(9), ASP(10)
# Masked residue at index 1 will be treated as GLY.
seq_1 = torch.tensor([[
    aa2num['LYS'], 21, aa2num['GLY'], aa2num['GLY'], aa2num['GLY'],
    aa2num['ALA'], aa2num['ARG'], aa2num['GLY'], aa2num['GLY'], aa2num['GLY'], aa2num['ASP']
]], dtype=torch.long)

# Chain break between residue 7 and 8 (rf_index jumps from 7 to 10)
rf_index_1 = torch.tensor([[0, 1, 2, 3, 4, 5, 6, 7, 10, 11, 12]], dtype=torch.long)

# Adjacency: LYS(0) <-> ALA(5) and ARG(6) <-> ASP(10)
adj_1 = torch.zeros(B, L, L, dtype=torch.long)
adj_1[0, 0, 5] = 1; adj_1[0, 5, 0] = 1  # LYS(0) - ALA(5)
# adj_1[0, 6, 10] = 1; adj_1[0, 10, 6] = 1 # ARG(6) - ASP(10)

# Calculate distance matrix
dist_matrix_1 = get_dist_matrix(adj_1, seq_1, connections, rf_index_1, dmax=30)
print(dist_matrix_1)
# --- Verification for Test Case 1 ---
# Path LYS(0)-CA to ALA(5)-CA:
# Peptide path: 5 residues apart = 5 * 3 = 15 bonds
# Special path: LYS(CA-CB-CG-CD-CE-NZ) - (CB-CA)ALA. Dist = 5 + 1 + 1 = 7
# Expected: 7
print(f"Distance LYS(0)-ALA(5): Expected=7, Calculated={dist_matrix_1[0, 0, 5].item()}")

# Path ALA(5)-CA to ARG(6)-CA:
# Peptide path: ALA(CA-C)-(N-CA)ARG. Dist = 1 + 1 + 1 = 3
# Expected: 3
print(f"Distance ALA(5)-ARG(6): Expected=3, Calculated={dist_matrix_1[0, 5, 6].item()}")

# Path ARG(6)-CA to ASP(10)-CA:
# Peptide path: ARG(6) to GLY(7) is connected (3 bonds). GLY(7) to GLY(8) is broken.
# So peptide path is infinite (or dmax).
# Special path: ARG(CA-CB-CG-CD-NE-CZ-NH1) - (CG-CB-CA)ASP. Dist = 6 + 1 + 2 = 9
# Expected: 9
print(f"Distance ARG(6)-ASP(10): Expected=9, Calculated={dist_matrix_1[0, 6, 10].item()}")

# Path GLY(7)-CA to GLY(8)-CA:
# No peptide bond due to rf_index break. No special connection.
# Expected: dmax (30)
print(f"Distance GLY(7)-GLY(8) across break: Expected=30, Calculated={dist_matrix_1[0, 7, 8].item()}")
print("\n")


# =============================================================================
# --- Test Case 2: N-terminus / C-terminus connection ---
# =============================================================================
print("--- Test Case 2: N-terminus / C-terminus connection ---")
B, L = 1, 15
# Sequence: Two separate dipeptides: LYS(0)-ALA(1) and ARG(2)-ASP(3)
seq_2 = torch.tensor([[aa2num['LYS'], aa2num['ALA'],21,21,21,21,21,21,21,21,21,aa2num['ASP'],21,21 
                       ,aa2num['ARG']]], dtype=torch.long)
# rf_index defines the two separate chains: {0, 1} and {10, 11}
rf_index_2 = torch.tensor([[0, 1,2,3,4,5,6,7, 10, 11,12,13,14,15,16]], dtype=torch.long)

# Adjacency: Connect the two chains. LYS(0) <-> ASP(3)
adj_2 = torch.zeros(B, L, L, dtype=torch.long)
adj_2[0, 0, -3] = 1; adj_2[0, -3, 0] = 1

# Specify that LYS(0) must connect via its N-terminus and ASP(3) via its C-terminus
N_connect_idx = None#torch.tensor([0], dtype=torch.long)
C_connect_idx = None#torch.tensor([14], dtype=torch.long)

# Calculate distance matrix
dist_matrix_2 = get_dist_matrix(adj_2, seq_2, connections, rf_index_2, dmax=30,
                                N_connect_idx=N_connect_idx, C_connect_idx=C_connect_idx)
print(dist_matrix_2)
# --- Verification for Test Case 2 ---
# Path LYS(0)-CA to ASP(3)-CA:
# LYS(0) is an N-terminus, ASP(3) is a C-terminus.
# Path: LYS(CA-N) - (C-CA)ASP. Dist = 1 + 1 + 1 = 3
# Expected: 3
print(f"Distance LYS(0)-ASP(3) via Termini: Expected=3, Calculated={dist_matrix_2[0, 0, 3].item()}")
print("\n")


# =============================================================================
# --- Test Case 3: Invalid backbone connection (should be ignored) ---
# =============================================================================
print("--- Test Case 3: Invalid backbone connection ---")
B, L = 1, 4
# Sequence: A single tetrapeptide
seq_3 = torch.tensor([[aa2num['LYS'], aa2num['ALA'], aa2num['ARG'], aa2num['ASP']]], dtype=torch.long)
rf_index_3 = torch.tensor([[0, 1, 2, 3]], dtype=torch.long)

# Adjacency: Try to connect LYS(0) and ARG(2) via N/C atoms
adj_3 = torch.zeros(B, L, L, dtype=torch.long)
adj_3[0, 0, 2] = 1; adj_3[0, 2, 0] = 1

# Specify that LYS(0) connects via N, but ARG(2) connects via C.
# LYS(0) is a valid N-terminus.
# ARG(2) is NOT a C-terminus, so this connection should be invalid and ignored.
N_connect_idx_3 = torch.tensor([0], dtype=torch.long)
C_connect_idx_3 = torch.tensor([2], dtype=torch.long)

# Calculate distance matrix
dist_matrix_3 = get_dist_matrix(adj_3, seq_3, connections, rf_index_3, dmax=30,
                                N_connect_idx=N_connect_idx_3, C_connect_idx=C_connect_idx_3)

# --- Verification for Test Case 3 ---
# Path LYS(0)-CA to ARG(2)-CA:
# The special connection is invalid because ARG(2) is not a C-terminus.
# The code should ignore it and fall back to the peptide path.
# Peptide path: LYS(0)-ALA(1)-ARG(2). 2 residues apart = 2 * 3 = 6 bonds.
# Expected: 6
print(f"Distance LYS(0)-ARG(2) with invalid C-term conn: Expected=6, Calculated={dist_matrix_3[0, 0, 2].item()}")
print(time.time() - start_time)

{(0, 15): [('C', 'OG', 'backbone')], (15, 0): [('OG', 'C', 'backbone')], (1, 3): [('N', 'CG', 'backbone')], (3, 1): [('CG', 'N', 'backbone')], (1, 6): [('NH2', 'OE1', 'sidechain')], (6, 1): [('OE1', 'NH2', 'sidechain')], (2, 11): [('CG', 'NZ', 'sidechain')], (11, 2): [('NZ', 'CG', 'sidechain')], (2, 15): [('CG', 'OG', 'sidechain')], (15, 2): [('OG', 'CG', 'sidechain')], (3, 4): [('CB', 'SG', 'sidechain'), ('C', 'SG', 'backbone')], (4, 3): [('SG', 'CB', 'sidechain'), ('SG', 'C', 'backbone')], (3, 7): [('CG', 'N', 'backbone')], (7, 3): [('N', 'CG', 'backbone')], (3, 11): [('CG', 'NZ', 'sidechain'), ('C', 'NZ', 'backbone')], (11, 3): [('NZ', 'CG', 'sidechain'), ('NZ', 'C', 'backbone')], (3, 16): [('CG', 'OG1', 'sidechain')], (16, 3): [('OG1', 'CG', 'sidechain')], (4, 4): [('SG', 'SG', 'sidechain'), ('SG', 'SG', 'sidechain')], (4, 6): [('SG', 'CG', 'sidechain')], (6, 4): [('CG', 'SG', 'sidechain')], (4, 7): [('SG', 'C', 'backbone')], (7, 4): [('C', 'SG', 'backbone')], (4, 8): [('SG', 'CE1'

In [15]:
import torch
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import shortest_path

def get_residue_dist_matrix(adj, rf_index, dmax=128):
    """
    Computes the residue-level shortest path distance matrix.

    The graph is constructed with residues as nodes. Edges exist between
    adjacent residues (peptide bonds) and between residues with special
    connections specified in the `adj` matrix. The distance is the
    minimum number of residues in the path.

    Args:
        adj (torch.Tensor): The adjacency matrix indicating special connections
                            (e.g., disulfide bonds). Shape: (B, L, L).
        rf_index (torch.Tensor): Residue indices from the PDB, used to identify
                                 sequential residues for peptide bonds.
                                 Shape: (B, L).
        dmax (int): The maximum distance to consider. Paths longer than this
                    will be capped at this value.

    Returns:
        torch.Tensor: A tensor of shape (B, L, L) containing the shortest
                      path distances between each pair of residues.
    """
    B, L = adj.shape[:2]
    # Initialize the final distance matrix with the maximum value
    dist_matrix = torch.full((B, L, L), float(dmax), dtype=torch.float32, device=adj.device)

    for b in range(B):
        # The nodes of our graph are the residues, so there are L nodes.
        if L == 0:
            continue

        row, col = [], []

        # Helper to add a bi-directional edge to the graph
        def add_edge(u, v):
            row.extend([u, v])
            col.extend([v, u])

        # 1. Add edges for peptide bonds (adjacent residues)
        for i in range(L - 1):
            # Check if residue i and i+1 are sequential in the chain
            print(i)
            print(rf_index[b, i])
            if  rf_index[b, i+1] == rf_index[b, i] + 1:
                add_edge(i, i + 1)

        # 2. Add edges for special connections from the input adjacency matrix
        # These could be disulfide bonds, cyclic connections, etc.
        special_connections = torch.argwhere(adj[b] == 1)
        for conn in special_connections:
            i, j = conn[0].item(), conn[1].item()
            # Avoid duplicate edges and self-loops
            if i < j:
                add_edge(i, j)
        
        # If there are no connections at all, fill with dmax and continue
        if not row:
            dist_matrix[b].fill_(dmax)
            dist_matrix[b].fill_diagonal_(0)
            continue

        # 3. Build a sparse matrix and calculate all-pairs shortest paths
        # The graph is unweighted, so each edge has a weight of 1.
        adj_matrix_sparse = csr_matrix(([1] * len(row), (row, col)), shape=(L, L))
        
        # Calculate shortest paths. Unreachable nodes will have a distance of 'inf'.
        residue_dist = shortest_path(csgraph=adj_matrix_sparse, directed=False, unweighted=True)
        
        # 4. Populate the final distance matrix for this batch item
        residue_dist_tensor = torch.from_numpy(residue_dist).to(dtype=torch.float32, device=adj.device)
        
        # Clamp the infinite distances (unreachable pairs) to dmax
        dist_matrix[b] = torch.clamp(residue_dist_tensor, max=dmax)

    return dist_matrix



In [17]:
B=2
L=11
adj_1 = torch.zeros(B, L, L, dtype=torch.long)
adj_1[0, 0, 5] = 1; adj_1[0, 5, 0] = 1  # LYS(0) - ALA(5)
rf_index_1 = torch.tensor([[0, 1, 2, 3, 4, 5, 6, 7, 10, 11, 12]], dtype=torch.long).repeat(B, 1)
mat = get_residue_dist_matrix(adj_1, rf_index_1, dmax=32)

0
tensor(0)
1
tensor(1)
2
tensor(2)
3
tensor(3)
4
tensor(4)
5
tensor(5)
6
tensor(6)
7
tensor(7)
8
tensor(10)
9
tensor(11)
0
tensor(0)
1
tensor(1)
2
tensor(2)
3
tensor(3)
4
tensor(4)
5
tensor(5)
6
tensor(6)
7
tensor(7)
8
tensor(10)
9
tensor(11)


In [18]:
mat

tensor([[[ 0.,  1.,  2.,  3.,  2.,  1.,  2.,  3., 32., 32., 32.],
         [ 1.,  0.,  1.,  2.,  3.,  2.,  3.,  4., 32., 32., 32.],
         [ 2.,  1.,  0.,  1.,  2.,  3.,  4.,  5., 32., 32., 32.],
         [ 3.,  2.,  1.,  0.,  1.,  2.,  3.,  4., 32., 32., 32.],
         [ 2.,  3.,  2.,  1.,  0.,  1.,  2.,  3., 32., 32., 32.],
         [ 1.,  2.,  3.,  2.,  1.,  0.,  1.,  2., 32., 32., 32.],
         [ 2.,  3.,  4.,  3.,  2.,  1.,  0.,  1., 32., 32., 32.],
         [ 3.,  4.,  5.,  4.,  3.,  2.,  1.,  0., 32., 32., 32.],
         [32., 32., 32., 32., 32., 32., 32., 32.,  0.,  1.,  2.],
         [32., 32., 32., 32., 32., 32., 32., 32.,  1.,  0.,  1.],
         [32., 32., 32., 32., 32., 32., 32., 32.,  2.,  1.,  0.]],

        [[ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7., 32., 32., 32.],
         [ 1.,  0.,  1.,  2.,  3.,  4.,  5.,  6., 32., 32., 32.],
         [ 2.,  1.,  0.,  1.,  2.,  3.,  4.,  5., 32., 32., 32.],
         [ 3.,  2.,  1.,  0.,  1.,  2.,  3.,  4., 32., 32., 32.],
        